<a href="https://colab.research.google.com/github/monbrandao/projeto-final/blob/main/ClinicalTrial_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

In [ ]:
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')

#nltk.download('wordnet')
from nltk.tokenize import word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/clinicaltrialsproject/records/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

# Importando arquivos XML

In [ ]:
import pandas as pd 
from glob import glob
import re

import xml.etree.ElementTree as et 

file_names=glob("*.xml")
cont = 0

dataset=[]
for file_name in file_names: 

  xtree = et.parse(file_name)
  
  xroot = xtree.getroot()
  text=xroot.find('TEXT').text.replace("\n",  '')
  text="{0:>18}".format(file_name) + ' - ' + re.sub(' +', ' ', text)
 # text= re.sub(' +', ' ', text)
  dataset.append(text) 
 

In [ ]:
record = pd.DataFrame(dataset, columns=['description'])

In [ ]:
def tp(descricao):
   
    if descricao.strip() == 'Record date':
        val = 'PR'  
    else:
        val = 'CT'
    return val

criar novas colunas no dataframe

In [ ]:
  record['file_name'] = record['description'].str[:18]
  
  #record['record_date'] = record['description'].str[:44].str[-10:]

  record['doc_type'] =  record['description'].str[:32].str[-12:].apply(tp)
 


In [ ]:
# corrigr campo data
record.head()

In [ ]:
record.shape

# Pré - Processamento de texto

In [ ]:

import re
import string

newline = lambda x: re.sub('\n', ' ', x) # remove \n
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x) # remove alphanumeric words
simbols = lambda x: re.sub('\W', ' ', x) # remove alphanumeric words
under = lambda x: re.sub('_', ' ', x) # remove hypen words
punc_re = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x) # remove puntuação
#inclui para tirar palavaras
lower_alpha = lambda x: re.sub(r"""\w*\d\w*""", ' ', x.lower())


#record['description']  = record.description.map(newline).map(lower_alpha).map(punc_re).map(under)
record['description']  = record.description.map(newline).map(under).map(simbols).map(alphanumeric).map(punc_re).map(lower_alpha)


In [ ]:
record['description']

# Tokenização

In [ ]:
record['tokens'] = record.description.map(word_tokenize)
record.head()

In [ ]:
word_list = sum(record.tokens.tolist(), [])

word_list[:10]

# Stopwords 

In [ ]:
# Estas são as etapas para encontrar as palavras mais comuns

from collections import Counter

# Converte a lista em um dicionário com contagem de valores
word_counts = Counter(word_list)

# Inverter a chave / valores no dicionário para classificarReverter a chave / valores no dicionário para classificar
word_counts = list(zip(word_counts.values(), word_counts.keys()))

# Classifique a lista por contagem
word_counts = sorted(word_counts, reverse=True)

# Imprime as 10 palavras mais comuns
word_counts[:10]

In [ ]:
def removeStops(texts, stopwords):
    texts_out = []
    for sent in texts:
        texts_out.append(" ".join([token for token in sent if token not in stopwords]))
    return texts_out


In [ ]:
# Remova as stop words
from nltk.corpus import stopwords
nltk.download("stopwords")

stop_words = stopwords.words('english')

stop_words += ["mg", "po", "qd", "dr", "p", "l", "pt", "xml", "ga", "date", "jr", "har","h","patient", "history", "pain", "tablet"]




# ... more stopwords

In [ ]:
stop_lambda = lambda x: [y for y in x if y not in stop_words]
record['tokens_stop'] = record.tokens.apply(stop_lambda)
record.head()

In [ ]:
word_list = sum(record.tokens_stop.tolist(), [])

word_list[:10]

In [ ]:
# partes de speech tagging
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')
pos_lambda = lambda x: nltk.pos_tag(x)
record['tokens_pos'] = (record.tokens_stop.apply(pos_lambda))
record.head()

In [ ]:

word_list_clean = sum(record.tokens_stop.tolist(), [])

word_list_clean[:10]

In [ ]:
from collections import Counter

# Converte a lista em um dicionário com contagem de valores
word_counts_clean = Counter(word_list_clean)
a = word_counts_clean

# Inverta a chave / valores no dicionário para classificar
word_counts_clean = list(zip(word_counts_clean.values(), word_counts_clean.keys()))

# Classifique a lista por contagem
word_counts_clean = sorted(word_counts_clean, reverse=True)

# Imprime as 10 palavras mais comuns
word_counts_clean[:10]

# Worldcloud

In [ ]:
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (10,10)) # Text that is not Fake
wc = WordCloud(background_color='white',max_words = 2000 , width = 1600 , height = 800 , stopwords = STOPWORDS).fit_words(a)
plt.imshow(wc , interpolation = 'bilinear')
plt.axis("off")
plt.show()



# Gerar o Corpus

In [ ]:
corpus = record.description
corpus.head()

In [ ]:
# Criada uma matriz de termos de documento usando o Count Vectorizer com as stopwords ativadas para inglês
from sklearn.feature_extraction.text import CountVectorizer
          
cv = CountVectorizer(stop_words="english",
                            analyzer='word',      
                             lowercase=True,)

term_frequencies = cv.fit_transform(corpus).toarray()
vocab = cv.get_feature_names ()




In [ ]:
dt = pd.DataFrame(term_frequencies, columns=cv.get_feature_names()).set_index(record.file_name) 
dt.head(9)

# Encontre o Registro mais parecido com o Clinical Trial NCT03986073  usando o TF-IDF Vectorizer (comparar documentos)

In [ ]:
record[record.file_name=='CT_NCT03986073.xml'].description

In [ ]:
clinicaltrial = list(dt.loc['CT_NCT03986073.xml'])
clinicaltrial[:20]

In [ ]:
# Definir o cálculo de similaridade de cosseno
from numpy import dot
from numpy.linalg import norm

cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

#Similiaridade do Cosseno

In [ ]:
similarity = [cosine(clinicaltrial, file_name) for file_name in term_frequencies]
sorted(list(zip(similarity, record.file_name)), reverse=True)[1:9]

#TF_IDF

In [ ]:
# Crie o vetorizador TF-IDF - stopwords pq faço uma comparação matematica e por isso preciso tirar as palavras que não quero
from sklearn.feature_extraction.text import TfidfVectorizer
          
cv_tfidf = TfidfVectorizer(stop_words="english")
X_tfidf = cv_tfidf.fit_transform(corpus).toarray()

dt_tfidf = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names()).set_index(record.file_name)
dt_tfidf.head()

In [ ]:
# Calcule todas as semelhanças e classifique as mais semelhantes usando os dados do TF-IDF
similarity_tfidf = [cosine(clinicaltrial, file_name) for file_name in X_tfidf]
sorted(list(zip(similarity_tfidf, record.file_name)), reverse=True)[1:9]

#LDA


In [ ]:
# cada letra de 
patient_record = [lyrics for lyrics in record.description] 
print("Temos %d registros de pacientes." %len(patient_record))

In [ ]:
!pip install pyLDAvis


In [ ]:
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint


import pyLDAvis
import pyLDAvis.sklearn


# remove stopwords and punctuations
def preprocess(row):
    return strip_punctuation(remove_stopwords(row.lower()))
    
record['description'] = record['description'] .apply(preprocess)

# Convert data to required input format by LDA
texts = []
for line in record.description:
    lowered = line.lower()
    words = re.findall(r'\w+', lowered, flags = re.UNICODE)# | re.LOCALE
    texts.append(words)
    
# Create a dictionary representation of the documents.
dictionary = Dictionary(texts)

# Filter out words that occur less than 2 documents, or more than 30% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.3)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(text) for text in texts]

## 2. Treinamos o modelo LDA

In [ ]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 50
iterations = 200
eval_every = None

# Train model
model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, chunksize=chunksize, alpha='auto', 
                          eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, 
                          eval_every=eval_every)

## 3. Distribuição de topicos de documentos
Usamos o método get_document_topics que realiza a infrencia de a distribuição de topicos de um documento. Retorna uma lista de tupa (topic_id, probability)

In [ ]:
# Get document topics
all_topics = model.get_document_topics(corpus, minimum_probability=0)
all_topics[2]

## 4. Preparando os arquivos de entrada para TensorBoard

In [ ]:
from tensorboard_helper import CreateTensorboardData

In [ ]:
vectors = [ [topics[1] for topics in doc_topics] for doc_topics in all_topics]
metadatos = [record.file_name.values, record.doc_type.values ]


In [ ]:
CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["file_name","doc_type"])

**Visualizando usando PCA ...**

Agora, vamos anexar os tópicos com maior probabilidade (topic_id, topic_probability) ao título do documento, para explorar quais tópicos os domínios ou bordas do cluster pertencem de forma dominante. Para isso, precisamos sobrescrever o arquivo de metadados conforme abaixo:

In [ ]:
tensors = []
for doc_topics in all_topics:
    doc_tensor = []
    for topic in doc_topics:
        if round(topic[1], 3) > 0:
            doc_tensor.append((topic[0], float(round(topic[1], 3))))
            
    # sort topics according to highest probabilities
    doc_tensor = sorted(doc_tensor, key=lambda x: x[1], reverse=True)
    # store vectors to add in metadata file
    tensors.append(doc_tensor[:5])

In [ ]:
tensors_str = [ str(" ".join([title, str(x)])) for title, x in zip(record.file_name.values, tensors)]
metadatos = [tensors_str, record.doc_type.values]

CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["file_name","doc_type"])

In [ ]:
model.show_topic(topicid=0, topn=10)

In [ ]:
import pyLDAvis.gensim

viz = pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(viz)

UsandoGridSearch para encontrar melhor modelo LDA
O parâmetro de ajuste mais importante para modelos LDA é n_components (número de tópicos). Além disso, vamos pesquisar learning_decay (que controla a taxa de aprendizado).

In [ ]:
vectorizer = CountVectorizer(analyzer='word',  
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{5,}',  # num chars > 5
                            )

# data_vectorized = vectorizer.fit_transform(data_lemmatized)
data_vectorized = vectorizer.fit_transform(record['description'])

In [ ]:
# Define Search Param
search_params = {'n_components': [5, 10, 15], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

#(model, corpus, dictionary)
#(lda_model, data_vectorized, vectorizer) 

Escolhendo o melhor modelo

In [ ]:
# Melhor modelo
best_lda_model = model.best_estimator_

# Hiperparâmetros do modelo
print("Melhores parâmetros: ", model.best_params_)

# probabilidade logarítmica
print("Melhor score de probabilidade logarítmica: ", model.best_score_)

# Perplexidade
print("Perplexidade do modelo: ", best_lda_model.perplexity(data_vectorized))

#comparando o score

In [ ]:
import seaborn as sns

In [ ]:
results = pd.DataFrame(model.cv_results_)

current_palette = sns.color_palette("Set2", 3)

plt.figure(figsize=(12,8))

sns.lineplot(data=results,
             x='param_n_components',
             y='mean_test_score',
             hue='param_learning_decay',
             palette=current_palette,
             marker='o'
            )

plt.show()

In [ ]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(patient_record))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics_first10 = df_document_topic[:10].style.applymap(color_green).applymap(make_bold)
df_document_topics_first10

#Quantidade de documento por tópico

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

#Visualizando LDA com pyLDA

In [ ]:
pyLDAvis.enable_notebook() 
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne') 
pyLDAvis.display(panel)

# Principais palavras por tópico

In [ ]:
vocab = vectorizer.get_feature_names()

# data_vectorized
topic_words = {}
n_top_words = 20

for topic, comp in enumerate(best_lda_model.components_):
    # for the n-dimensional array "arr":
    # argsort() returns a ranked n-dimensional array of arr, call it "ranked_array"
    # which contains the indices that would sort arr in a descending fashion
    # for the ith element in ranked_array, ranked_array[i] represents the index of the
    # element in arr that should be at the ith index in ranked_array
    # ex. arr = [3,7,1,0,3,6]
    # np.argsort(arr) -> [3, 2, 0, 4, 5, 1]
    # word_idx contains the indices in "topic" of the top num_top_words most relevant
    # to a given topic ... it is sorted ascending to begin with and then reversed (desc. now)    
    word_idx = np.argsort(comp)[::-1][:n_top_words]

    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

In [ ]:
# tranformando objeto style em um dataframe pandas
df2 = pd.DataFrame(data=df_document_topics.data, columns=df_document_topics.columns)
df2.head(15)

In [ ]:
df2["file_name"] = record["file_name"].tolist()
df2.tail(10)

In [ ]:
df2[df2["dominant_topic"]==2].groupby(["file_name"]).size().sort_values(ascending=False)[:10]

In [ ]:
df2[df2["dominant_topic"]==2].sort_values(by=['Topic2'], ascending=False)[:15]

In [ ]:


# Print the top 10 words per topic
n_words = 10
feature_names = vectorizer.get_feature_names()

topic_list = []
for topic_idx, topic in enumerate(best_lda_model.components_):
    top_n = [feature_names[i]
             for i in topic.argsort()
             [-n_words:]][::-1]
    top_features = ' '.join(top_n)
    topic_list.append(f"topic_{'_'.join(top_n[:3])}") 

    print(f"Topic {topic_idx}: {top_features}")

In [ ]:
amounts = model.transform(data_vectorized) * 100

# Set it up as a dataframe
topics = pd.DataFrame(amounts, columns=topic_list)
topics.head(2)

#Doc2vec

In [ ]:
import gensim

In [ ]:
def read_corpus(documents):
    for i, plot in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot, max_len=30), [i])

In [ ]:
train_corpus = list(read_corpus(record.description))

## 2. Treinando o modelo doc2vec

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

#RANKEAR OS DOCUMENTOS


In [ ]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [ ]:
import collections

counter = collections.Counter(ranks)
print(counter)

In [ ]:
print( 'Clinical Trial - Document number {} : file_name - {} - «{}»\n'.format(doc_id,  record.file_name[doc_id] ,' '.join(train_corpus[doc_id].words)))
print(u'Similiarity of the documents per model using Word2ve %s:\n' % model)
for label, index in [('MOST SIMILAR', 1), ('SECOND-MOST SIMILAR', 2), ('THIRD-MOST SIMILAR', 3), ('MEDIAN', len(sims)//2), ('LEAST SIMILAR', len(sims) - 1)]:  
  print(u'* %s %s : file_name -%s-: «%s»\n' % (label, sims[index],  record.file_name[sims[index][0]].strip(), ' '.join(train_corpus[sims[index][0]].words)))
   

In [ ]:
tensor_file = 'doc_tensor.w2v'
model.save_word2vec_format(tensor_file, doctag_vec=True, word_vec=False)
key_vectors = gensim.models.KeyedVectors.load_word2vec_format(tensor_file, binary=False)

In [ ]:
descriptions_id = []
vectors = []
for description_id in key_vectors.index2word:
    descriptions_id.append(description_id)
    vectors.append(key_vectors[description_id])

In [ ]:
from tensorboard_helper import CreateTensorboardData

3. Preparando os arquivos de entrada para TensorBoard

In [ ]:
metadatos=[descriptions_id]
CreateTensorboardData(tensor_filename="patient_record", 
                      vectors=vectors, 
                      metadatos=[descriptions_id])

In [ ]:
metadatos=[record.file_name.values, record.doc_type.values]

CreateTensorboardData(tensor_filename="patient_record", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["file_name","doc_type"])